<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# HubSpot - Create contacts from linkedin post likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/HubSpot/HubSpot_Create_contacts_from_linkedin_post_likes.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Create+contacts+from+linkedin+post+likes:+Error+short+description">Bug report</a>

**Tags:** #hubspot #crm #sales #contact #naas_drivers #linkedin #post #contact #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-04-12 (Created: 2022-02-21)

**Description:** This notebook create new contacts based on LinkedIn post likes.

## Input

### Import librairies

In [ ]:
from naas_drivers import linkedin, hubspot
import naas
import requests

### Setup HubSpot
👉 Starting November 30, 2022, HubSpot API keys no longer enable access to HubSpot APIs, so in Naas version 2.8.3 and above, you need [create a private app and use the access token](https://developers.hubspot.com/docs/api/private-apps).

In [ ]:
# Enter Your Access Token
HS_ACCESS_TOKEN = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"

### Setup LinkedIn
If you are using the Chrome Extension:

- [Install Naas Chrome Extension](https://chrome.google.com/webstore/detail/naas/cpkgfedlkfiknjpkmhcglmjiefnechpp?hl=fr&authuser=0)
- [Create a new token](https://app.naas.ai/hub/token)
- Copy/Paste your token in your extension
- Login/Logout your LinkedIn account
- Your secrets "LINKEDIN_LI_AT" and "LINKEDIN_JSESSIONID" will be added directly on your naas everytime you login and logout.

or <br>

If you are not using the Google Chrome Extension, [learn how to get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75) and set up the values below:
- 🍪 li_at
- 🍪 JSESSIONID

In [ ]:
# Cookies
LI_AT = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_COOKIE_LI_AT"
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_COOKIE_JSESSIONID"

# Post URL
POST_URL = "----"

## Model

### Get post likes

In [ ]:
df_posts = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(POST_URL)

# Display the number of likes
print("Number of likes: ", df_posts.PROFILE_URN.count())

In [ ]:
# Show dataframe with list of profiles from likes
df_posts

### Create contacts from LinkedIn post likes

In [ ]:
def create_contacts_from_post(
    df,
    c_profile_urn="PROFILE_URN",
    c_firstname="FIRSTNAME",
    c_lastname="LASTNAME",
    c_occupation="OCCUPATION",
):
    for _, row in df.iterrows():
        profile_urn = row[c_profile_urn]
        firstname = row[c_firstname]
        lastname = row[c_lastname]
        occupation = row[c_occupation]
        linkedinbio = f"https://www.linkedin.com/in/{profile_urn}"
        email = None
        phone = None

        # contact
        try:
            contact = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(
                linkedinbio
            )
            email = contact.loc[0, "EMAIL"]
            phone = contact.loc[0, "PHONENUMBER"]
        except:
            print("No contact info")

        # With send method
        data = {
            "properties": {
                "linkedinbio": linkedinbio,
                "firstname": firstname,
                "lastname": lastname,
                "jobtitle": occupation,
                "email": email,
                "phone": phone,
            }
        }
        print(data)
        hubspot.connect(HS_ACCESS_TOKEN).contacts.send(data)

## Output

### Display result

In [ ]:
create_contacts_from_post(df_posts)